#**My 1st notebook for playing around with NOAA competition**

* red: adult males
* magenta: subadult males
* brown: adult females
* blue: juveniles
* green: pups

References :
[Kevin Mader][1], 
[Philipp Schmidt][2] 


  [1]: https://www.kaggle.com/kmader/noaa-fisheries-steller-sea-lion-population-count/qbi-2017-single-object-analysis
  [2]: https://www.kaggle.com/philschmidt/noaa-fisheries-steller-sea-lion-population-count/counting-sea-lions/run/1044523

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid': False})
from scipy import stats
from skimage.io import imread, imshow
from skimage.util import crop
import os
import cv2
from collections import namedtuple

In [ ]:
def get_dots_from_image(cropped_dotted, cropped_raw):
    """
    # Get the markers only
    There are also brown markers which are removed by our thresholding 
    and are also not very present in the difference image itself.
    """
    y_max, x_max, _ = cropped_dotted.shape
    diff = cv2.subtract(cropped_dotted, cropped_raw)
    diff = diff/diff.max()
    diff = cv2.absdiff(cropped_dotted, cropped_raw)
    gray = cv2.cvtColor(diff, cv2.COLOR_RGB2GRAY)
    ret,th1 = cv2.threshold(gray,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    cnts = cv2.findContours(th1.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
    x, y = [], []
    for loc in cnts:
        x.append(loc[0][0][0])
        y.append(loc[0][0][1])
    x = np.array(x)
    y = np.array(y)
    return x,y

labimg=namedtuple('LabeledImage',['image','x','y'])

def load_image_and_labels(img_id):
    """
    Read the images and compute the x,y coordinates of the sea lions
    """
    temp_dotted = cv2.cvtColor(cv2.imread('../input/TrainDotted/{}'.format(img_id)), cv2.COLOR_BGR2RGB)
    temp_raw = cv2.cvtColor(cv2.imread('../input/Train/{}'.format(img_id)), cv2.COLOR_BGR2RGB)
    x,y = get_dots_from_image(temp_dotted,temp_raw)
    return labimg(temp_raw,x,y)

#**Sample Images**

In [ ]:
training_image_ids = [os.path.basename(c) for c in glob('../input/Train/*.jpg')]

In [ ]:
for img_id in training_image_ids:
    temp_dotted = cv2.cvtColor(cv2.imread('../input/TrainDotted/{}'.format(img_id)), cv2.COLOR_BGR2RGB)
    temp_raw = cv2.cvtColor(cv2.imread('../input/Train/{}'.format(img_id)), cv2.COLOR_BGR2RGB)
    fig = plt.figure(figsize=(12,8))
    fig.suptitle(img_id)
    plt.subplot(121)
    plt.imshow(temp_raw)
    plt.xticks([]), plt.yticks([])
    plt.subplot(122)
    plt.imshow(temp_dotted)
    plt.xticks([]), plt.yticks([])
    plt.show()

##Experiments

In [ ]:
img_exp=5
exp_dotted = cv2.cvtColor(cv2.imread('../input/TrainDotted/{}.jpg'.format(img_exp)), cv2.COLOR_BGR2RGB)[1350:1900, 3000:3400]#[0:3230, 2800:3950]
exp_raw = cv2.cvtColor(cv2.imread('../input/Train/{}.jpg'.format(img_exp)), cv2.COLOR_BGR2RGB)[1350:1900, 3000:3400]#[0:3230, 2800:3950]
fig = plt.figure(figsize=(12,8))
fig.suptitle(img_exp)
plt.subplot(121)
plt.imshow(exp_raw)
plt.xticks([]), plt.yticks([])
plt.subplot(122)
plt.imshow(exp_dotted)
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
exp_raw.shape